https://www.acmicpc.net/problem/3665

팀은 1번부터 n번까지 번호가 매겨져 있다. 올해 인터넷 예선 본부에서는 최종 순위를 발표하지 않기로 했다. 그 대신에 작년에 비해 상대적으로 순위가 바뀐 팀의 목록만 발표하려고 한다. 예를 들어, 작년에 팀 13이 팀 6보다 순위가 높았는데, 올해 팀 6이 팀 13보다 순위가 높다면, (6,13)을 발표할 것이다.

이 정보만을 가지고 올해 최종 순위를 만들어보려고 하는데, 작년 순위와 상대적인 순위가 바뀐 모든 팀의 목록이 주어졌을 때, 올해 순위를 만드는 프로그램을 작성하시오. 하지만, 본부에서 발표한 정보를 가지고 확실한 올해 순위를 만들 수 없는 경우가 있을 수도 있고, 일관성이 없는 잘못된 정보일 수도 있다. 이 두 경우도 모두 찾아내야 한다.

입력 조건 )
- 첫째 줄에는 테스트 케이스의 개수가 주어진다. 테스트 케이스는 100개를 넘지 않는다. 각 테스트 케이스는 다음과 같다.
    * 팀의 수 n을 포함하는 한 줄 (2 <= n <= 500)
    * n개의 정수 t_i를 포함하고 있는 한 줄. (1 <= t_i <= n) t_i는 작년에 i등을 한 팀의 번호이다. 1등이 가장 성적이 높은 팀이다. 모든 t_i는 서로 다르다.
    * 상대적인 등수가 바뀐 쌍의 수 m (0 <= m <= 25000)
    * 두 정수 a_i와 b_i를 포함하고 있는 m 줄. (1 <= a_i < b_i <= n) 상대적인 등수가 바뀐 두 팀이 주어진다. 같은 쌍이 여러 번 발표되는 경우는 없다.

출력 조건 )
- 각 테스트 케이스에 대해 다음을 출력한다.
    * n개의 정수를 한 줄에 출력한다. 출력하는 숫자는 올해 순위이며, 1등팀부터 순서대로 출력한다. 만약, 확실한 순위를 찾을 수 없다면 '?'를 출력한다. 데이터에 일관성이 없어 순위를 정할 수 없는 경우에는 'IMPOSSIBLE'을 출력한다.

In [8]:
from collections import deque

t = int(input())
for _ in range(t) :
    n = int(input()) # 팀의 수
    rank = list(map(int, input().split())) # 작년 등수 (인덱스+1는 팀 번호)
    m = int(input()) # 작년과 다른 등수를 가진 팀의 수
    
    indegree = [0] * (n+1) # 진입차수 초기화
    graph = [[False] * (n+1) for _ in range(n+1)] # 연결된 간선 정보
    for i in range(n) :
        for j in range(i + 1, n) :
            graph[rank[i]][rank[j]] = True
            indegree[rank[j]] += 1
    
    for _ in range(m) :
        a, b = map(int, input().split())
        # 간선 방향 뒤집기
        if graph[a][b] :
            graph[a][b] = False
            graph[b][a] = True
            indegree[a] += 1
            indegree[b] -= 1
        else :
            graph[a][b] = True
            graph[b][a] = False
            indegree[a] -= 1
            indegree[b] += 1
    
    result = []
    q = deque()
    for i in range(1, n+1) :
        if indegree[i] == 0 :
            q.append(i)
    
    certain = True # 위상 정렬 결과가 오직 하나인지
    cycle = False # 그래프 내 사이클이 존재하는지
    
    # 정확히 노드의 개수만큼 반복
    for i in range(n) :
        # 큐가 비어있다면 사이클 발생
        if len(q) == 0 :
            cycle = True
            break
        # 큐의 원소가 2개 이상이라면 가능한 정렬 결과가 여러 개
        if len(q) >= 2 :
            certain = False
            break
        
        now = q.popleft()
        result.append(now)
        # 해당 원소와 연결된 노드들의 진입차수에서 1 빼기
        for j in range(1, n+1) :
            if graph[now][j] :
                indegree[j] -= 1
                # 새롭게 진입차수가 0이 되는 노드를 큐에 삽입
                if indegree[j] == 0 :
                    q.append(j)
                    
    # 사이클 발생하는 경우(일관성 없는 경우)
    if cycle :
        print('IMPOSSIBLE')
    # 위상 정렬 결과 여러 개인 경우
    elif not certain :
        print('?')
    # 위상 정렬 수행한 결과
    else :
        for i in result :
            print(i, end=' ')
        print()

3
5
5 4 3 2 1
2 
2 4
3 4
5 3 2 4 1 
3
2 3 1
0
2 3 1 
4
1 2 3 4
3
1 2
3 4
2 3
IMPOSSIBLE


In [12]:
n=5
rank =[5,4,3,2,1]
indegree = [0] * (n+1) # 진입차수 초기화
graph = [[False] * (n+1) for _ in range(n+1)] # 연결된 간선 정보
for i in range(n) :
    for j in range(i + 1, n) :
        graph[rank[i]][rank[j]] = True
        indegree[rank[j]] += 1

In [13]:
indegree

[0, 4, 3, 2, 1, 0]

In [14]:
graph

[[False, False, False, False, False, False],
 [False, False, False, False, False, False],
 [False, True, False, False, False, False],
 [False, True, True, False, False, False],
 [False, True, True, True, False, False],
 [False, True, True, True, True, False]]

In [15]:
for i in range(n) :
    for j in range(i+1, n) :
        print(i, j)

0 1
0 2
0 3
0 4
1 2
1 3
1 4
2 3
2 4
3 4


In [16]:
m = 2
for _ in range(m) :
        a, b = map(int, input().split())
        # 간선 방향 뒤집기
        if graph[a][b] :
            graph[a][b] = False
            graph[b][a] = True
            indegree[a] += 1
            indegree[b] -= 1
        else :
            graph[a][b] = True
            graph[b][a] = False
            indegree[a] -= 1
            indegree[b] += 1

2 4
3 4


In [18]:
graph, indegree

([[False, False, False, False, False, False],
  [False, False, False, False, False, False],
  [False, True, False, False, True, False],
  [False, True, True, False, True, False],
  [False, True, False, False, False, False],
  [False, True, True, True, True, False]],
 [0, 4, 2, 1, 3, 0])

In [19]:
result = []
q = deque()
for i in range(1, n+1) :
    if indegree[i] == 0 :
        q.append(i)

certain = True # 위상 정렬 결과가 오직 하나인지
cycle = False # 그래프 내 사이클이 존재하는지

# 정확히 노드의 개수만큼 반복
for i in range(n) :
    # 큐가 비어있다면 사이클 발생
    if len(q) == 0 :
        cycle = True
        break
    # 큐의 원소가 2개 이상이라면 가능한 정렬 결과가 여러 개
    if len(q) >= 2 :
        certain = False
        break

    now = q.popleft()
    result.append(now)
    # 해당 원소와 연결된 노드들의 진입차수에서 1 빼기
    for j in range(1, n+1) :
        if graph[now][j] :
            indegree[j] -= 1
            # 새롭게 진입차수가 0이 되는 노드를 큐에 삽입
            if indegree[j] == 0 :
                q.append(j)

In [20]:
result

[5, 3, 2, 4, 1]